In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=545e4c47bfbb1990de344157943c75820c1b90e06ee4fe34ac336ec05bfac150
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np
import math
import re
from sklearn.cluster import KMeans
from itertools import islice
from scipy.spatial import distance
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import itertools
#take n first items
def take(n, iterable):
    return list(islice(iterable, n))

def extract_importance_sentence(content, max_length):
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    sentences = content.split(".")
    for i in range (len(sentences)):
        sentences[i] = sentences[i].strip()
        embeddings = model.encode(sentences)
    #print(embeddings)
    #print(len(sentences))
    fixed_params = {"max_iter": 300}

    param_grid = {"n_clusters": range(2, len(sentences)//2), 
                  "n_init": [10], 
                  "random_state": [0, 100, 200], 
                  "algorithm": ["lloyd"]}

    best_silhouette_score = -1
    best_hyperparameters = {}
    centers = None
    #scores = []

    for params in itertools.product(*param_grid.values()):
        # create a dictionary of hyperparameters by combining param_grid with fixed_params
        params_dict = dict(zip(param_grid.keys(), params))
        params_dict.update(fixed_params) 

        # create a KMeans object with the hyperparameters specified in params_dict
        kmeans = KMeans(**params_dict)

        # fit the KMeans object to the embeddings
        kmeans.fit(embeddings)

        # calculate the silhouette score for the clustering results
        labels = kmeans.labels_
        score = silhouette_score(embeddings, labels, metric="euclidean")
        #print(score)
        #scores.append(score)
        # check if the current hyperparameters produce a better silhouette score
        if score > best_silhouette_score:
            best_silhouette_score = score
            best_hyperparameters = params_dict
            centers = kmeans.cluster_centers_
    mean_distances = {}
    for i in range(len(embeddings)):
        name_embedding = 'embedding_' + str(i)
        distances = []
        for center in centers:
            distance_  = distance.euclidean(np.array(embeddings[i]), center)
            distances.append(distance_)
        mean_distances[name_embedding] = np.mean(distances)
    #print(mean_distances)
    sorted_mean_distances = dict(sorted(mean_distances.items(), key = lambda item: item[1]))
    #print(sorted_mean_distances)
    selected_embeddings = take(len(sentences), sorted_mean_distances.items())
    #print(selected_embeddings)
    selected_index = []

    for selected_embedding in selected_embeddings:
        temp_list=selected_embedding[0].split('_')
        #print(temp_list[-1])
        index_chosen = int(temp_list[-1])
        selected_index.append(index_chosen)
    #print(selected_index)
  
    selected_list_sentences = []
    length_sentences_chosen = 0
    for index in selected_index.copy():
        if len(tokenizer(sentences[index])["input_ids"]) > max_length:
            selected_index.remove(index)
    final_selected_index = []
    for index in selected_index:
        length_sentences_chosen += len(tokenizer(sentences[index])["input_ids"])
        if length_sentences_chosen < max_length :
            final_selected_index.append(index)
    #print(final_selected_index)
    sorted_final_selected_index = sorted(final_selected_index)
    for index in sorted_final_selected_index:
        selected_list_sentences.append(sentences[index])
    important_document = '. '.join(selected_list_sentences)
    important_document_strip = important_document.strip()
    if len(important_document_strip)!= 0:
        if important_document_strip[-1] != '.':
            final_important_document = '. '.join(selected_list_sentences) + '.'
        else:
            final_important_document = important_document_strip
        return final_important_document
    else:
        return content


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
def is_longer_than_512(document):
    if len(tokenizer(document)["input_ids"]) > 512:
        return True
    else:
        return False
def is_longer_than_1024(document):
    if len(tokenizer(document)["input_ids"]) > 1024:
        return True
    else:
        return False

In [4]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 62.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 41.1 MB/s eta 0:00:00


In [5]:
from underthesea import pos_tag

def add_prompt(text):
    words = pos_tag(text)
    txt = ""
    for word in words:
        if word[1] in ["N", "V", "A"]:
            txt += word[0] + " {} ".format(word[1])
       
        else:
            txt += word[0] + " "
    return txt

In [6]:
import pandas as pd

In [7]:
df1 = pd.read_csv('/kaggle/input/nlpdataset/bio_medicine.csv')
df2 = pd.read_csv('/kaggle/input/nlpdataset/bio_medicine.csv')
#df1.head()

In [9]:
df1['Document_1024_Kmeans'] = df1['Document']
df1['Document_512_Kmeans'] = df1['Document']
df1['Document_512_Kmeans_token'] = [None] * len(df1['Summary'])
for i in range(len(df1["Summary"])):
    if is_longer_than_1024(df2['Document'][i]):
        if extract_importance_sentence(df2['Document'][i], 1024) is not None:
            df1['Document_1024_Kmeans'][i] = extract_importance_sentence(df2['Document'][i], 1024)
    if is_longer_than_512(df2['Document'][i]):
        if extract_importance_sentence(df2['Document'][i], 512) is not None:
            df1['Document_512_Kmeans'][i] = extract_importance_sentence(df2['Document'][i], 512)
  
    df1["Document_512_Kmeans_token"][i] = add_prompt(df1["Document_512_Kmeans"][i])

3790


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

3791


In [ ]:
df1.to_csv('/kaggle/working/bio_medicine_kmeans_full_new.csv') 